In [ ]:
#global planner without u turn

import time
import numpy as np


map=world.get_map()
def key_exists(key, dict_of_dicts):
    for i in dict_of_dicts:a
        if key in list(i.keys()):
            return True
    return False

waypoint_tuple_list = map.get_topology()
id_map = dict()
id_map_v = dict()
road_id_to_edge = dict()
graph={}
for waypoint in waypoint_tuple_list:
    x1, y1, x2, y2 = np.round([waypoint[0].transform.location.x, waypoint[0].transform.location.y,waypoint[1].transform.location.x, waypoint[1].transform.location.y], 0)
    road_id, section_id, lane_id = waypoint[0].road_id, waypoint[0].section_id, waypoint[0].lane_id
    if (x1,y1) not in id_map:
        id_map[(x1,y1)] = len(id_map)
        id_map_v[len(id_map_v)]=(x1,y1)
    if (x2,y2) not in id_map:
        id_map[(x2,y2)] = len(id_map)
        id_map_v[len(id_map_v)]=(x2,y2)
    if id_map[(x1,y1)] not in graph :
        graph[id_map[(x1,y1)]]={id_map[(x2,y2)]:waypoint[0]}
    else:
        if id_map[(x2,y2)] not in graph[id_map[(x1,y1)]]:
            graph[id_map[(x1,y1)]].update({id_map[(x2,y2)]:waypoint[0]})
    n1 = id_map[(x1,y1)]
    n2 = id_map[(x2,y2)]
    if road_id not in road_id_to_edge:
        road_id_to_edge[road_id] = dict()
    if section_id not in road_id_to_edge[road_id]:
        road_id_to_edge[road_id][section_id] = dict()
    road_id_to_edge[road_id][section_id][lane_id] = (n1, n2)

def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def get_path(start,end):
    wa_start=map.get_waypoint(start)
    road_start_id, section_start_id, lane_start_id = wa_start.road_id, wa_start.section_id, wa_start.lane_id
    (n1_start, n2_start)=road_id_to_edge[road_start_id][section_start_id][lane_start_id] 
    
    wa_end=map.get_waypoint(end)
    road_end_id, section_end_id, lane_end_id = wa_end.road_id, wa_end.section_id, wa_end.lane_id
    (n1_end, n2_end)=road_id_to_edge[road_end_id][section_end_id][lane_end_id]

    rout=[]
    if((n1_end, n2_end)!=(n1_start, n2_start)) or dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])<dist([wa_start.next(1)[0].transform.location.x,wa_start.next(1)[0].transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y]):
        (x1,y1),(x2,y2) = id_map_v[n2_start],id_map_v[n1_end]
        g = len(wa_start.next_until_lane_end(1))
        f=g+dist([x1,y1],[x2,y2])
        open = [[f,g,dist([x1,y1],[x2,y2]), x1, y1]]
        found = False 
        f_distance={node:float('inf') for node in graph}
        f_distance[n2_start]=f
    
        came_from={node:None for node in graph}
        came_from[n2_start]=n2_start
        #came_from={'s':None}
        #came_from={'su':None}
        #came_from={'n':None}
        #came_from={'nu':None}
        #came_from[n2_start]='s'
        
        while not found:
            if len(open) == 0:
                return "Fail"
                break
            else:
                min_ele = open[0][0]
                index=0
                for i in range(1, len(open)):
                    if open[i][0] < min_ele:
                        min_ele = open[i][0]
                        index=i
                next=open.pop(index)
                x = next[3]
                y = next[4]
                g = next[1]
                for i in graph[id_map[(x,y)]]:
                    (xf,yf)=id_map_v[i]
                    g2=len(graph[id_map[(x,y)]][i].next_until_lane_end(1))+g
                    f2=g2+dist([xf,yf],[x2,y2])
                    if came_from[i]==None:
                        open.append([f2,g2,dist([xf,yf],[x2,y2]), xf, yf])
                        f_distance[id_map[(xf,yf)]]=f2
                        came_from[id_map[(xf,yf)]]=id_map[(x,y)]
                        
                        if xf == x2 and yf == y2:
                            found = True
                            #came_from['n']=id_map[(x2,y2)]
                            #return came_from
                            wa=map.get_waypoint(carla.Location(x=x2,y=y2,z=0))
                            rout.append(wa)
                            while dist([wa.transform.location.x,wa.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
                                wa=wa.next(1)[0]
                                rout.append(wa)
                            while n1_end != n2_start:
                                n=came_from[n1_end]
                                ws=graph[n][n1_end].next_until_lane_end(1)
                                ws.extend(rout)
                                rout=ws.copy()
                                n1_end=n
                            ws=map.get_waypoint(carla.Location(x=wa_start.transform.location.x,y=wa_start.transform.location.y,z=0)).next_until_lane_end(1)
                            ws.extend(rout)
                            rout=ws.copy()
                            break
    else:
        rout.append(wa_start)
        while dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
            wa_start=wa_start.next(1)[0]
            rout.append(wa_start)
        rout.append(wa_end)
    return rout


In [ ]:
#global planner with u turn

import time
import numpy as np


map=world.get_map()
def key_exists(key, dict_of_dicts):
    for i in dict_of_dicts:
        if key in list(i.keys()):
            return True
    return False

waypoint_tuple_list = map.get_topology()
id_map = dict()
id_map_v = dict()
road_id_to_edge = dict()
graph={}
for waypoint in waypoint_tuple_list:
    x1, y1, x2, y2 = np.round([waypoint[0].transform.location.x, waypoint[0].transform.location.y,waypoint[1].transform.location.x, waypoint[1].transform.location.y], 0)
    road_id, section_id, lane_id = waypoint[0].road_id, waypoint[0].section_id, waypoint[0].lane_id
    if (x1,y1) not in id_map:
        id_map[(x1,y1)] = len(id_map)
        id_map_v[len(id_map_v)]=(x1,y1)
    if (x2,y2) not in id_map:
        id_map[(x2,y2)] = len(id_map)
        id_map_v[len(id_map_v)]=(x2,y2)
    if id_map[(x1,y1)] not in graph :
        graph[id_map[(x1,y1)]]={id_map[(x2,y2)]:waypoint[0]}
    else:
        if id_map[(x2,y2)] not in graph[id_map[(x1,y1)]]:
            graph[id_map[(x1,y1)]].update({id_map[(x2,y2)]:waypoint[0]})
    n1 = id_map[(x1,y1)]
    n2 = id_map[(x2,y2)]
    if road_id not in road_id_to_edge:
        road_id_to_edge[road_id] = dict()
    if section_id not in road_id_to_edge[road_id]:
        road_id_to_edge[road_id][section_id] = dict()
    road_id_to_edge[road_id][section_id][lane_id] = (n1, n2)

def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def get_path(start,end):
    wa_start=map.get_waypoint(start)
    road_start_id, section_start_id, lane_start_id = wa_start.road_id, wa_start.section_id, wa_start.lane_id
    (n1_start, n2_start)=road_id_to_edge[road_start_id][section_start_id][lane_start_id] 
    
    wa_end=map.get_waypoint(end)
    road_end_id, section_end_id, lane_end_id = wa_end.road_id, wa_end.section_id, wa_end.lane_id
    (n1_end, n2_end)=road_id_to_edge[road_end_id][section_end_id][lane_end_id]

 #   graph_c=graph.copy()
#    id_map_c=id_map.copy()
#    graph_c['s']={n2_start:wa_start}
    wsl=wa_start.get_left_lane()
    road_start_id, section_start_id, lane_start_id = wsl.road_id, wsl.section_id, wsl.lane_id
    (nu1_start, nu2_start)=road_id_to_edge[road_start_id][section_start_id][lane_start_id]
#    graph_c['su']={nu2_start:wsl}
#    graph_c[n1_end].update({'n':graph_c[n1_end][n2_end]})
    wnl=wa_end.get_left_lane()
    road_start_id, section_start_id, lane_start_id = wnl.road_id, wnl.section_id, wnl.lane_id
    (nu1_end, nu2_end)=road_id_to_edge[road_start_id][section_start_id][lane_start_id]
#    graph_c[nu1_end].update({'nu':graph_c[nu1_end][nu2_end]})
#    graph_c[nu1_end].update({'nu':graph_c[nu1_end][nu2_end]})
    rout=[]
    (x1,y1),(x2,y2) = id_map_v[n2_start],id_map_v[n1_end]
    (x3,y3),(x4,y4) = id_map_v[nu2_start],id_map_v[nu1_end]
    g1 = len(wa_start.next_until_lane_end(1))
    g2 = len(wsl.next_until_lane_end(1))
    h1=dist([x1,y1],[x2,y2])
    if h1>dist([x1,y1],[x4,y4]):
        h1=dist([x1,y1],[x4,y4])

    h2=dist([x3,y3],[x2,y2])
    if h2>dist([x3,y3],[x4,y4]):
        h2=dist([x3,y3],[x4,y4])
        
    f1=g1+h1
    f2=g2+h2
    open = [[f1,g1,h1,['s',n2_start]]]
    open = [[f2,g2,h2,['su',nu2_start]]]
    found = False 
    f_distance={node:float('inf') for node in graph}
    f_distance[n2_start]=f1

    came_from1={node:None for node in graph}
    came_from1[n2_start]=n2_start

    fu_distance={node:float('inf') for node in graph}
    fu_distance[nu2_start]=f2

    came_from_u={node:None for node in graph}
    came_from_u[nu2_start]=nu2_start

    if((nu1_end, nu2_end)==(n1_start, n2_start)) and dist([wa_start.transform.location.x,wa_start.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>dist([wa_start.next(1)[0].transform.location.x,wa_start.next(1)[0].transform.location.y],[wnl.transform.location.x,wnl.transform.location.y]):

        rout.append(wa_start)
        while dist([wa_start.transform.location.x,wa_start.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>1:
            wa_start=wa_start.next(1)[0]
            rout.append(wa_start)
        rout.append(wa_end)
        return rout

    elif((n1_end, n2_end)==(nu1_start, nu2_start)) and dist([wsl.transform.location.x,wsl.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>dist([wsl.next(1)[0].transform.location.x,wsl.next(1)[0].transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y]):

        rout.append(wa_start)
        while dist([wsl.transform.location.x,wsl.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
            rout.append(wsl)
            wsl=wsl.next(1)[0]
        rout.append(wa_end)
        return rout

    elif((nu1_end, nu2_end)==(nu1_start, nu2_start)) and dist([wsl.transform.location.x,wsl.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>dist([wsl.next(1)[0].transform.location.x,wsl.next(1)[0].transform.location.y],[wnl.transform.location.x,wnl.transform.location.y]):

        rout.append(wa_start)
        while dist([wsl.transform.location.x,wsl.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>1:
            rout.append(wsl)
            wsl=wsl.next(1)[0]
        rout.append(wa_end)
        return rout

    elif((n1_end, n2_end)!=(n1_start, n2_start)) or dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])<dist([wa_start.next(1)[0].transform.location.x,wa_start.next(1)[0].transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y]):

        while not found:
            if len(open) == 0:
                return "Fail"
                break
            else:
                min_ele = open[0][0]
                index=0
                for i in range(1, len(open)):
                    if open[i][0] < min_ele:
                        min_ele = open[i][0]
                        index=i
                next=open.pop(index)
                path = next[3].copy()
                #print(path[-1])
                g = next[1]
                for i in graph[path[-1]]:
                    #print(i)
                    (xf,yf)=id_map_v[i]
                    g2=len(graph[path[-1]][i].next_until_lane_end(1))+g
                    #print(g2)
                    h=dist([xf,yf],[x2,y2])
                    if h>dist([xf,yf],[x4,y4]):
                        h=dist([xf,yf],[x4,y4])
                    f2=g2+h
                    if path[0]=='s':
                        came_from=came_from1
                    elif path[0]=='su':
                        came_from=came_from_u
                    if came_from[i]==None:
                        f_distance[id_map[(xf,yf)]]=f2
                        came_from[id_map[(xf,yf)]]=path[-1]
                        fpath=path.copy()
                        fpath.append(i)
                        open.append([f2,g2,h,fpath])
                        
                        if xf == x2 and yf == y2:
                            found = True
                            #came_from['n']=id_map[(x2,y2)]
                            #return came_from
                            wa=map.get_waypoint(carla.Location(x=x2,y=y2,z=0))
                            rout.append(wa)
                            while dist([wa.transform.location.x,wa.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
                                wa=wa.next(1)[0]
                                rout.append(wa)

                            if path[0]=='su':
                                n2_start=nu2_start
                            while n1_end != n2_start:
                                n=came_from[n1_end]
                                ws=graph[n][n1_end].next_until_lane_end(1)
                                ws.extend(rout)
                                rout=ws.copy()
                                n1_end=n
                            if path[0]=='su':
                                ws=[wa_start]
                                ws.extend(wsl.next_until_lane_end(1))
                            elif path[0]=='s':
                                ws=wa_start.next_until_lane_end(1)
                            ws.extend(rout)
                            rout=ws.copy()
                            break

                        elif xf == x4 and yf == y4:
                            found = True
                            #came_from['n']=id_map[(x2,y2)]
                            #return came_from
                            wa=map.get_waypoint(carla.Location(x=x4,y=y4,z=0))
                            rout.append(wa)
                            while dist([wa.transform.location.x,wa.transform.location.y],[wnl.transform.location.x,wnl.transform.location.y])>1:
                                wa=wa.next(1)[0]
                                rout.append(wa)

                            rout.append(wa_end)
                            if path[0]=='su':
                                n2_start=nu2_start
                            while nu1_end != n2_start:
                                n=came_from[nu1_end]
                                ws=graph[n][nu1_end].next_until_lane_end(1)
                                ws.extend(rout)
                                rout=ws.copy()
                                nu1_end=n
                            if path[0]=='su':
                                ws=[wa_start]
                                ws.extend(wsl.next_until_lane_end(1))
                            elif path[0]=='s':
                                ws=wa_start.next_until_lane_end(1)
                            ws.extend(rout)
                            rout=ws.copy()
                            break
    else:
        rout.append(wa_start)
        while dist([wa_start.transform.location.x,wa_start.transform.location.y],[wa_end.transform.location.x,wa_end.transform.location.y])>1:
            wa_start=wa_start.next(1)[0]
            rout.append(wa_start)
        rout.append(wa_end)
    return rout


points=[[-8,140],[100,103],[193,125],[193,200],[165,235],[80,239],[135,210],[40,210],[40,290],[20,190],[95,305],[2,300]]

start=carla.Location(x=points[0][0],y=points[0][1],z=0)
world.debug.draw_string(start, 's', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)
end=carla.Location(x=points[1][0],y=points[1][1],z=0)
world.debug.draw_string(end, 'e', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=60.0,
            persistent_lines=True)
route=get_path(start,end)
#print(route)
for i in route:
    world.debug.draw_string(i.transform.location, '*', draw_shadow=False,
            color=carla.Color(r=0, g=255, b=255), life_time=60.0,
            persistent_lines=True)
    #time.sleep(1)
#end=map.get_waypoint(end)
#road_end_id, section_end_id, lane_end_id = end.road_id, end.section_id, end.lane_id
#(n1, n2)=road_id_to_edge[road_end_id][section_end_id][lane_end_id]

#print(n1)


In [ ]:
#algorithm
import sys
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)
def dist(s,e):
    return abs(s[0]-e[0])+abs(s[1]-e[1])
def astar():
    graph=[[i for i in range(12)]for j in range(12)]
    points=[[-8,140],[100,103],[193,125],[193,200],[165,235],[80,239],[135,210],[40,210],[40,290],[20,190],[95,305],[2,300]]
    start=[193,230]
    for i in range(12):
        for j in range(12):
            if i != j:
                #if (i==4 and j==3)or(i==6 and j==3)or(i==7 and j==0)or(i==8 and j==0)or(i==9 and j==0)or(i==11 and j==0):
                #    continue
                print(f"from {i} to {j}:")
                print(len(get_path(carla.Location(x=points[i][0],y=points[i][1],z=0), carla.Location(x=points[j][0],y=points[j][1],z=0))))
    f=0
    h=0
    g=0
    open=[[f,g,h,[start]]]
    path=[]
    while True:
        min_ele = open[0][0]
        index=0
        for i in range(1, len(open)):
            if open[i][0] < min_ele:
                min_ele = open[i][0]
                index=i
        next = open[index]
        open.pop(index)
        path=next[3].copy()
        for i in range(len(points)):
            if points[i] not in path:
                gp=len(get_path(carla.Location(x=path[len(path)-1][0],y=path[len(path)-1][1],z=0), carla.Location(x=points[i][0],y=points[i][1],z=0)))
                if (gp<450 and path[len(path)-1]==[95,305]) or(gp<300 and path[len(path)-1]!=[95,305]):
                    g=gp+next[1]
                    h=0
                    fpath=path.copy()
                    fpath.append(points[i])
                    for j in range(len(points)):
                        if points[j]not in fpath:
                            hj=dist(points[j],points[i])
                            #hj=len(get_path(carla.Location(x=points[i][0],y=points[i][1],z=0), carla.Location(x=points[j][0],y=points[j][1],z=0)))

                            if hj>h:
                                h=hj
                    f=2.1*h+g
                    open.append([f,g,h,fpath])
                    if len(fpath)==len(points)+1:
                        return fpath
astar()

In [ ]:
#add all routes together
ws1=get_path(carla.Location(x=193,y=230,z=0),carla.Location(x=193,y=200,z=0))
ws2=get_path(carla.Location(x=193,y=200,z=0),carla.Location(x=193,y=125,z=0))
ws3=get_path(carla.Location(x=193,y=125,z=0),carla.Location(x=100,y=103,z=0))
ws4=get_path(carla.Location(x=100,y=103,z=0),carla.Location(x=-8,y=140,z=0))
ws5=get_path(carla.Location(x=-8,y=140,z=0),carla.Location(x=20,y=190,z=0))
ws6=get_path(carla.Location(x=20,y=190,z=0),carla.Location(x=40,y=210,z=0))
ws7=get_path(carla.Location(x=40,y=210,z=0),carla.Location(x=80,y=239,z=0))
ws8=get_path(carla.Location(x=80,y=239,z=0),carla.Location(x=135,y=210,z=0))
ws9=get_path(carla.Location(x=135,y=210,z=0),carla.Location(x=40,y=290,z=0))
ws11=get_path(carla.Location(x=40,y=290,z=0),carla.Location(x=95,y=305,z=0))
ws12=get_path(carla.Location(x=95,y=305,z=0),carla.Location(x=165,y=235,z=0))
ws10=get_path(carla.Location(x=165,y=235,z=0),carla.Location(x=2,y=300,z=0))
wss=[ws1,ws2,ws3,ws4,ws5,ws6,ws7,ws8,ws9,ws10,ws11,ws12]
route=[]
for ws in wss:
    route.extend(ws)
    for w in ws:
        world.debug.draw_string(w.transform.location, '^', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=255), life_time=600.0,
            persistent_lines=True)
print(len(route))

In [ ]:
#get sensor point cloud (ros)
from __future__ import print_function



from collections import namedtuple
import ctypes
import math
import struct

import roslib.message
from sensor_msgs.msg import PointCloud2, PointField

_DATATYPES = {}
_DATATYPES[PointField.INT8]    = ('b', 1)
_DATATYPES[PointField.UINT8]   = ('B', 1)
_DATATYPES[PointField.INT16]   = ('h', 2)
_DATATYPES[PointField.UINT16]  = ('H', 2)
_DATATYPES[PointField.INT32]   = ('i', 4)
_DATATYPES[PointField.UINT32]  = ('I', 4)
_DATATYPES[PointField.FLOAT32] = ('f', 4)
_DATATYPES[PointField.FLOAT64] = ('d', 8)

def read_points(cloud, field_names=None, skip_nans=False, uvs=[]):

    assert isinstance(cloud, roslib.message.Message) and cloud._type == 'sensor_msgs/PointCloud2', 'cloud is not a sensor_msgs.msg.PointCloud2'
    fmt = _get_struct_fmt(cloud.is_bigendian, cloud.fields, field_names)
    width, height, point_step, row_step, data, isnan = cloud.width, cloud.height, cloud.point_step, cloud.row_step, cloud.data, math.isnan
    unpack_from = struct.Struct(fmt).unpack_from

    if skip_nans:
        
        for v in range(height):
            offset = row_step * v
            for u in range(width):
                p = unpack_from(data, offset)
                has_nan = False
                for pv in p:
                    if isnan(pv):
                        has_nan = True
                        break
                if not has_nan:
                    yield p
                offset += point_step
    

def _get_struct_fmt(is_bigendian, fields, field_names=None):
    fmt = '>' if is_bigendian else '<'

    offset = 0
    for field in (f for f in sorted(fields, key=lambda f: f.offset) if field_names is None or f.name in field_names):
        if offset < field.offset:
            fmt += 'x' * (field.offset - offset)
            offset = field.offset
        if field.datatype not in _DATATYPES:
            print('Skipping unknown PointField datatype [%d]' % field.datatype, file=sys.stderr)
        else:
            datatype_fmt, datatype_length = _DATATYPES[field.datatype]
            fmt    += field.count * datatype_fmt
            offset += field.count * datatype_length

    return fmt

In [ ]:
#car follow the path
import carla #the sim library itself
import cv2 #to work with images from cameras
import time # towork with images from cameras
import numpy as np #in this example to change image representation - re-shaping
import math
import random
import sys
#astar()
client = carla.Client('localhost', 2000)
#world = client.get_world()
world = client.load_world('Town02')
map=world.get_map()
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)

ws1=grp.trace_route(carla.Location(x=193,y=230,z=0),carla.Location(x=193,y=200,z=0))
ws2=grp.trace_route(carla.Location(x=193,y=200,z=0),carla.Location(x=193,y=125,z=0))
wa=map.get_waypoint(carla.Location(x=100,y=103,z=0))
ws3=grp.trace_route(carla.Location(x=193,y=125,z=0),wa.transform.location)
ws4=grp.trace_route(carla.Location(x=100,y=103,z=0),carla.Location(x=-8,y=140,z=0))
ws5=grp.trace_route(carla.Location(x=-8,y=140,z=0),carla.Location(x=20,y=190,z=0))
ws6=grp.trace_route(carla.Location(x=20,y=190,z=0),carla.Location(x=40,y=210,z=0))
ws7=grp.trace_route(carla.Location(x=40,y=210,z=0),carla.Location(x=80,y=239,z=0))
ws8=grp.trace_route(carla.Location(x=80,y=239,z=0),carla.Location(x=135,y=210,z=0))
wa=map.get_waypoint(carla.Location(x=165,y=235,z=0))
ws9=grp.trace_route(carla.Location(x=135,y=210,z=0),wa.transform.location)
ws10=grp.trace_route(carla.Location(x=165,y=235,z=0),carla.Location(x=40,y=290,z=0))
ws11=grp.trace_route(carla.Location(x=40,y=290,z=0),carla.Location(x=95,y=305,z=0))
ws12=grp.trace_route(carla.Location(x=95,y=305,z=0),carla.Location(x=2,y=300,z=0))
wss=[ws1,ws2,ws3,ws4,ws5,ws6,ws7,ws8,ws9,ws10,ws11,ws12]
route=[]
for ws in wss:
    route.extend(ws)
    for w in ws:
        world.debug.draw_string(w[0].transform.location, '^', draw_shadow=False,
            color=carla.Color(r=255, g=0, b=255), life_time=600.0,
            persistent_lines=True)

# define speed contstants
PREFERRED_SPEED = 30 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 40

#camera mount offset on the car - you can tweak these to have the car in view or not
CAMERA_POS_Z = 3 
CAMERA_POS_X = -5 

#adding params to display text to image
font = cv2.FONT_HERSHEY_SIMPLEX
# org - defining lines to display telemetry values on the screen
org = (30, 30) # this line will be used to show current speed
org2 = (30, 50) # this line will be used for future steering angle
org3 = (30, 70) # and another line for future telemetry outputs
org4 = (30, 90) # and another line for future telemetry outputs
org3 = (30, 110) # and another line for future telemetry outputs
fontScale = 0.5
# white color
color = (255, 255, 255)
# Line thickness of 2 px
thickness = 1


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

#function to subtract 2 vectors
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.transform.location.x
    wp_y = wp.transform.location.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

#spawn_points = world.get_map().get_spawn_points()
start_point = route[0][0].transform
start_point.location.z=0.5
#spectator = world.get_spectator() #start_point was used to spawn the car but we move 1m up to avoid being on the floor
#spawn_points[0].location.y = 230
#spawn_points[0].location.x = 193
#spawn_points[0].rotation.pitch=-3
#spawn_points[0].rotation.yaw=3
#spawn_points[0].rotation.roll=-3
#look for a blueprint of Mini car
vehicle_bp = world.get_blueprint_library().filter('*mini*')

vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)

#setting RGB Camera - this follow the approach explained in a Carla video
# link: https://www.youtube.com/watch?v=om8klsBj4rc&t=1184s

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640') # this ratio works in CARLA 9.14 on Windows
camera_bp.set_attribute('image_size_y', '360')

camera_init_trans = carla.Transform(carla.Location(z=CAMERA_POS_Z,x=CAMERA_POS_X))
#this creates the camera in the sim
camera = world.spawn_actor(camera_bp,camera_init_trans,attach_to=vehicle)

def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))

image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()

camera_data = {'image': np.zeros((image_h,image_w,4))}
# this actually opens a live stream from the camera
camera.listen(lambda image: camera_callback(image,camera_data))

cv2.namedWindow('RGB Camera',cv2.WINDOW_AUTOSIZE)
cv2.imshow('RGB Camera',camera_data['image'])

#main loop 
quit = False
curr_wp = 5 #we will be tracking waypoints in the route and switch to next one wen we get close to current one
predicted_angle = 0
while curr_wp<len(route)-1:
    # Carla Tick
    world.tick()
    if cv2.waitKey(1) == ord('q'):
        quit = True
        vehicle.apply_control(carla.VehicleControl(throttle=0,steer=0,brake=1))
        break
    image = camera_data['image']
    
    while curr_wp<len(route) and vehicle.get_transform().location.distance(route[curr_wp][0].transform.location)<5:
        curr_wp +=1 #move to next wp if we are too close
    
    predicted_angle = get_angle(vehicle,route[curr_wp][0])
    image = cv2.putText(image, 'Steering angle: '+str(round(predicted_angle,3)), org, font, fontScale, color, thickness, cv2.LINE_AA)
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    image = cv2.putText(image, 'Speed: '+str(int(speed)), org2, font, fontScale, color, thickness, cv2.LINE_AA)
    image = cv2.putText(image, 'Next wp: '+str(curr_wp), org3, font, fontScale, color, thickness, cv2.LINE_AA)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75

    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
    cv2.imshow('RGB Camera',image)
    

#clean up
cv2.destroyAllWindows()
camera.stop()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()

In [ ]:
#spline
import numpy as np
import scipy.optimize
import scipy.integrate
from math import sin, cos, pi, sqrt
from matplotlib import pyplot as plt

class PathOptimizer:
    def __init__(self):
        self._xf = 0.0
        self._yf = 0.0
        self._tf = 0.0

    def optimize_spiral(self, xf, yf, tf):
        self._xf = xf
        self._yf = yf
        self._tf = tf
        sf_0 = np.linalg.norm([xf, yf])
        p0 = [0.0, 0.0, sf_0]
        bounds = scipy.optimize.Bounds([-0.5, -0.5, sf_0], [0.5, 0.5, np.inf])
        res = scipy.optimize.minimize(self.objective, p0, method = 'L-BFGS-B', jac = self.objective_grad, bounds = bounds)
        spiral = self.sample_spiral(res.x)
        return spiral


    def thetaf(self, a, b, c, d, s):
        thetas = [a * x + b * x**2 / 2 + c * x**3 / 3 + d * x**4 / 4 for x in s]
        return thetas

    def sample_spiral(self, p):
        p = [0.0, p[0], p[1], 0.0, p[2]]    # recall p0 and p3 are set to 0
                                            # and p4 is the final arc length
        a = p[0]
        b = -(11.0*p[0]/2.0 - 9.0*p[1] + 9.0*p[2]/2.0 - p[3])/p[4]
        c = (9.0*p[0] - 45.0*p[1]/2.0 + 18.0*p[2] - 9.0*p[3]/2.0)/p[4]**2
        d = -(9.0*p[0]/2.0 - 27.0*p[1]/2.0 + 27.0*p[2]/2.0 - 9.0*p[3]/2.0)/p[4]**3


        s_points = np.linspace(0.0, p[4])


        t_points = self.thetaf(a, b, c, d, s_points)
        x_points = scipy.integrate.cumtrapz(np.cos(t_points), s_points, initial=0.0)
        y_points = scipy.integrate.cumtrapz(np.sin(t_points), s_points, initial=0.0)
        return [x_points, y_points, t_points]


    def objective(self, p):
        p = [0.0, p[0], p[1], 0.0, p[2]]
        return self.fbe(p) + 25*(self.fxf(p) + self.fyf(p)) + 30*self.ftf(p)

    def objective_grad(self, p):
        p = [0.0, p[0], p[1], 0.0, p[2]]
        return np.add(np.add(np.add(self.fbe_grad(p), np.multiply(25, self.fxf_grad(p))),np.multiply(25, self.fyf_grad(p))), np.multiply(30, self.ftf_grad(p)))

    def fxf(self, p):
        t2 = p[0]*(1.1E1/2.0);
        t3 = p[1]*9.0;
        t4 = p[2]*(9.0/2.0);
        t5 = p[0]*(9.0/2.0);
        t6 = p[1]*(2.7E1/2.0);
        t7 = p[2]*(2.7E1/2.0);
        t8 = p[3]*(9.0/2.0);
        t9 = t5-t6+t7-t8;
        t10 = p[0]*9.0;
        t11 = p[1]*(4.5E1/2.0);
        t12 = p[2]*1.8E1;
        t13 = t8-t10+t11-t12;
        t14 = p[3]-t2+t3-t4;
        t15 = self._xf-p[4]*(cos(p[0]*p[4]-p[4]*t9*(1.0/4.0)-p[4]*t13*(1.0/3.0)+p[4]*t14*(1.0/2.0))+cos(p[0]*p[4]*(1.0/2.0)-p[4]*t9*(1.0/6.4E1)-p[4]*t13*(1.0/2.4E1)+p[4]*t14*(1.0/8.0))*2.0+cos(p[0]*p[4]*(3.0/4.0)-p[4]*t9*7.91015625E-2-p[4]*t13*(9.0/6.4E1)+p[4]*t14*(9.0/3.2E1))*2.0+cos(p[0]*p[4]*(1.0/4.0)-p[4]*t9*9.765625E-4-p[4]*t13*(1.0/1.92E2)+p[4]*t14*(1.0/3.2E1))*2.0+cos(p[0]*p[4]*(3.0/8.0)-p[4]*t9*4.94384765625E-3-p[4]*t13*(9.0/5.12E2)+p[4]*t14*(9.0/1.28E2))*4.0+cos(p[0]*p[4]*(1.0/8.0)-p[4]*t9*6.103515625E-5-p[4]*t13*6.510416666666667E-4+p[4]*t14*(1.0/1.28E2))*4.0+cos(p[0]*p[4]*(5.0/8.0)-p[4]*t9*3.814697265625E-2-p[4]*t13*8.138020833333333E-2+p[4]*t14*(2.5E1/1.28E2))*4.0+cos(p[0]*p[4]*(7.0/8.0)-p[4]*t9*1.4654541015625E-1-p[4]*t13*2.233072916666667E-1+p[4]*t14*(4.9E1/1.28E2))*4.0+1.0)*(1.0/2.4E1);
        t0 = t15*t15;
        return t0

    def fxf_grad(self, p):
        grad = [0.0, 0.0, 0.0]

        t2 = p[0]*(1.1E1/2.0);
        t3 = p[1]*9.0;
        t4 = p[2]*(9.0/2.0);
        t5 = p[0]*(9.0/2.0);
        t6 = p[1]*(2.7E1/2.0);
        t7 = p[2]*(2.7E1/2.0);
        t8 = p[3]*(9.0/2.0);
        t9 = t5-t6+t7-t8;
        t10 = p[0]*9.0;
        t11 = p[1]*(4.5E1/2.0);
        t12 = p[2]*1.8E1;
        t13 = t8-t10+t11-t12;
        t14 = p[3]-t2+t3-t4;
        t15 = p[0]*p[4];
        t16 = p[0]*p[4]*(1.0/2.0);
        t17 = p[0]*p[4]*(3.0/4.0);
        t18 = p[0]*p[4]*(1.0/4.0);
        t19 = p[0]*p[4]*(3.0/8.0);
        t20 = p[0]*p[4]*(1.0/8.0);
        t21 = p[0]*p[4]*(5.0/8.0);
        t22 = p[0]*p[4]*(7.0/8.0);
        t0 = p[4]*(self._xf-p[4]*(cos(t15-p[4]*t9*(1.0/4.0)-p[4]*t13*(1.0/3.0)+p[4]*t14*(1.0/2.0))+cos(t16-p[4]*t9*(1.0/6.4E1)-p[4]*t13*(1.0/2.4E1)+p[4]*t14*(1.0/8.0))*2.0+cos(t17-p[4]*t9*7.91015625E-2-p[4]*t13*(9.0/6.4E1)+p[4]*t14*(9.0/3.2E1))*2.0+cos(t18-p[4]*t9*9.765625E-4-p[4]*t13*(1.0/1.92E2)+p[4]*t14*(1.0/3.2E1))*2.0+cos(t19-p[4]*t9*4.94384765625E-3-p[4]*t13*(9.0/5.12E2)+p[4]*t14*(9.0/1.28E2))*4.0+cos(t20-p[4]*t9*6.103515625E-5-p[4]*t13*6.510416666666667E-4+p[4]*t14*(1.0/1.28E2))*4.0+cos(t21-p[4]*t9*3.814697265625E-2-p[4]*t13*8.138020833333333E-2+p[4]*t14*(2.5E1/1.28E2))*4.0+cos(t22-p[4]*t9*1.4654541015625E-1-p[4]*t13*2.233072916666667E-1+p[4]*t14*(4.9E1/1.28E2))*4.0+1.0)*(1.0/2.4E1))*(p[4]*sin(t15-p[4]*t9*(1.0/4.0)-p[4]*t13*(1.0/3.0)+p[4]*(p[3]-t2+t3-t4)*(1.0/2.0))*(3.0/8.0)+p[4]*sin(t16-p[4]*t9*(1.0/6.4E1)-p[4]*t13*(1.0/2.4E1)+p[4]*(p[3]-t2+t3-t4)*(1.0/8.0))*(5.1E1/6.4E1)+p[4]*sin(t17-p[4]*t9*7.91015625E-2-p[4]*t13*(9.0/6.4E1)+p[4]*(p[3]-t2+t3-t4)*(9.0/3.2E1))*8.701171875E-1+p[4]*sin(t18-p[4]*t9*9.765625E-4-p[4]*t13*(1.0/1.92E2)+p[4]*(p[3]-t2+t3-t4)*(1.0/3.2E1))*3.544921875E-1+p[4]*sin(t19-p[4]*t9*4.94384765625E-3-p[4]*t13*(9.0/5.12E2)+p[4]*(p[3]-t2+t3-t4)*(9.0/1.28E2))*1.2161865234375+p[4]*sin(t20-p[4]*t9*6.103515625E-5-p[4]*t13*6.510416666666667E-4+p[4]*(p[3]-t2+t3-t4)*(1.0/1.28E2))*2.259521484375E-1+p[4]*sin(t21-p[4]*t9*3.814697265625E-2-p[4]*t13*8.138020833333333E-2+p[4]*(p[3]-t2+t3-t4)*(2.5E1/1.28E2))*1.7669677734375+p[4]*sin(t22-p[4]*t9*1.4654541015625E-1-p[4]*t13*2.233072916666667E-1+p[4]*(p[3]-t2+t3-t4)*(4.9E1/1.28E2))*1.5970458984375)*(1.0/1.2E1);
        grad[0] = t0

        t2 = p[0]*(1.1E1/2.0);
        t3 = p[1]*9.0;
        t4 = p[2]*(9.0/2.0);
        t5 = p[0]*(9.0/2.0);
        t6 = p[1]*(2.7E1/2.0);
        t7 = p[2]*(2.7E1/2.0);
        t8 = p[3]*(9.0/2.0);
        t9 = t5-t6+t7-t8;
        t10 = p[0]*9.0;
        t11 = p[1]*(4.5E1/2.0);
        t12 = p[2]*1.8E1;
        t13 = t8-t10+t11-t12;
        t14 = p[3]-t2+t3-t4;
        t15 = p[0]*p[4];
        t16 = p[0]*p[4]*(1.0/2.0);
        t17 = p[4]*t14*(1.0/8.0);
        t18 = t16+t17-p[4]*t9*(1.0/6.4E1)-p[4]*t13*(1.0/2.4E1);
        t19 = p[0]*p[4]*(3.0/4.0);
        t20 = p[0]*p[4]*(1.0/4.0);
        t21 = p[4]*t14*(1.0/3.2E1);
        t22 = t20+t21-p[4]*t9*9.765625E-4-p[4]*t13*(1.0/1.92E2);
        t23 = p[0]*p[4]*(3.0/8.0);
        t24 = p[4]*t14*(9.0/1.28E2);
        t25 = t23+t24-p[4]*t9*4.94384765625E-3-p[4]*t13*(9.0/5.12E2);
        t26 = p[0]*p[4]*(1.0/8.0);
        t27 = p[4]*t14*(1.0/1.28E2);
        t28 = t26+t27-p[4]*t9*6.103515625E-5-p[4]*t13*6.510416666666667E-4;
        t29 = p[0]*p[4]*(5.0/8.0);
        t30 = p[0]*p[4]*(7.0/8.0);
        t0 = p[4]*(self._xf-p[4]*(cos(t15-p[4]*t9*(1.0/4.0)-p[4]*t13*(1.0/3.0)+p[4]*t14*(1.0/2.0))+cos(t19-p[4]*t9*7.91015625E-2-p[4]*t13*(9.0/6.4E1)+p[4]*t14*(9.0/3.2E1))*2.0+cos(t29-p[4]*t9*3.814697265625E-2-p[4]*t13*8.138020833333333E-2+p[4]*t14*(2.5E1/1.28E2))*4.0+cos(t30-p[4]*t9*1.4654541015625E-1-p[4]*t13*2.233072916666667E-1+p[4]*t14*(4.9E1/1.28E2))*4.0+cos(t18)*2.0+cos(t22)*2.0+cos(t25)*4.0+cos(t28)*4.0+1.0)*(1.0/2.4E1))*(p[4]*sin(t15-p[4]*t9*(1.0/4.0)-p[4]*t13*(1.0/3.0)+p[4]*(p[3]-t2+t3-t4)*(1.0/2.0))*(3.0/8.0)+p[4]*sin(t19-p[4]*t9*7.91015625E-2-p[4]*t13*(9.0/6.4E1)+p[4]*(p[3]-t2+t3-t4)*(9.0/3.2E1))*3.955078125E-1+p[4]*sin(t29-p[4]*t9*3.814697265625E-2-p[4]*t13*8.138020833333333E-2+p[4]*(p[3]-t2+t3-t4)*(2.5E1/1.28E2))*2.838134765625E-1+p[4]*sin(t30-p[4]*t9*1.4654541015625E-1-p[4]*t13*2.233072916666667E-1+p[4]*(p[3]-t2+t3-t4)*(4.9E1/1.28E2))*1.2740478515625-p[4]*sin(t18)*(3.0/6.4E1)-p[4]*sin(t22)*1.201171875E-1-p[4]*sin(t25)*2.669677734375E-1-p[4]*sin(t28)*9.70458984375E-2)*(1.0/1.2E1);
        grad[1] = t0

        t2 = p[0]*(1.1E1/2.0);
        t3 = p[1]*9.0;
        t4 = p[2]*(9.0/2.0);
        t5 = p[0]*(9.0/2.0);
        t6 = p[1]*(2.7E1/2.0);
        t7 = p[2]*(2.7E1/2.0);
        t8 = p[3]*(9.0/2.0);
        t9 = t5-t6+t7-t8;
        t10 = p[0]*9.0;
        t11 = p[1]*(4.5E1/2.0);
        t12 = p[2]*1.8E1;
        t13 = t8-t10+t11-t12;
        t14 = p[3]-t2+t3-t4;
        t15 = p[0]*p[4];
        t16 = p[0]*p[4]*(1.0/2.0);
        t17 = p[0]*p[4]*(3.0/4.0);
        t18 = p[0]*p[4]*(1.0/4.0);
        t19 = p[0]*p[4]*(3.0/8.0);
        t20 = p[0]*p[4]*(1.0/8.0);
        t21 = p[0]*p[4]*(5.0/8.0);
        t22 = p[0]*p[4]*(7.0/8.0);
        t23 = p[4]*(p[3]-t2+t3-t4)*(1.0/2.0);
        t39 = p[4]*t9*(1.0/4.0);
        t40 = p[4]*t13*(1.0/3.0);
        t24 = t15+t23-t39-t40;
        t25 = p[4]*(p[3]-t2+t3-t4)*(1.0/8.0);
        t41 = p[4]*t9*(1.0/6.4E1);
        t42 = p[4]*t13*(1.0/2.4E1);
        t26 = t16+t25-t41-t42;
        t27 = p[4]*(p[3]-t2+t3-t4)*(1.0/3.2E1);
        t45 = p[4]*t9*9.765625E-4;
        t46 = p[4]*t13*(1.0/1.92E2);
        t28 = t18+t27-t45-t46;
        t29 = p[4]*(p[3]-t2+t3-t4)*(9.0/3.2E1);
        t43 = p[4]*t9*7.91015625E-2;
        t44 = p[4]*t13*(9.0/6.4E1);
        t30 = t17+t29-t43-t44;
        t31 = p[4]*(p[3]-t2+t3-t4)*(1.0/1.28E2);
        t49 = p[4]*t9*6.103515625E-5;
        t50 = p[4]*t13*6.510416666666667E-4;
        t32 = t20+t31-t49-t50;
        t33 = p[4]*(p[3]-t2+t3-t4)*(9.0/1.28E2);
        t47 = p[4]*t9*4.94384765625E-3;
        t48 = p[4]*t13*(9.0/5.12E2);
        t34 = t19+t33-t47-t48;
        t35 = p[4]*(p[3]-t2+t3-t4)*(2.5E1/1.28E2);
        t51 = p[4]*t9*3.814697265625E-2;
        t52 = p[4]*t13*8.138020833333333E-2;
        t36 = t21+t35-t51-t52;
        t37 = p[4]*(p[3]-t2+t3-t4)*(4.9E1/1.28E2);
        t53 = p[4]*t9*1.4654541015625E-1;
        t54 = p[4]*t13*2.233072916666667E-1;
        t38 = t22+t37-t53-t54;
        t0 = (self._xf-p[4]*(cos(t15-t39-t40+p[4]*t14*(1.0/2.0))+cos(t16-t41-t42+p[4]*t14*(1.0/8.0))*2.0+cos(t18-t45-t46+p[4]*t14*(1.0/3.2E1))*2.0+cos(t17-t43-t44+p[4]*t14*(9.0/3.2E1))*2.0+cos(t20-t49-t50+p[4]*t14*(1.0/1.28E2))*4.0+cos(t19-t47-t48+p[4]*t14*(9.0/1.28E2))*4.0+cos(t21-t51-t52+p[4]*t14*(2.5E1/1.28E2))*4.0+cos(t22-t53-t54+p[4]*t14*(4.9E1/1.28E2))*4.0+1.0)*(1.0/2.4E1))*(cos(t24)*(1.0/2.4E1)+cos(t26)*(1.0/1.2E1)+cos(t28)*(1.0/1.2E1)+cos(t30)*(1.0/1.2E1)+cos(t32)*(1.0/6.0)+cos(t34)*(1.0/6.0)+cos(t36)*(1.0/6.0)+cos(t38)*(1.0/6.0)-p[4]*(sin(t24)*(p[0]*(1.0/8.0)+p[1]*(3.0/8.0)+p[2]*(3.0/8.0)+p[3]*(1.0/8.0))+sin(t26)*(p[0]*(1.5E1/1.28E2)+p[1]*(5.1E1/1.28E2)-p[2]*(3.0/1.28E2)+p[3]*(1.0/1.28E2))*2.0+sin(t28)*(p[0]*1.2060546875E-1+p[1]*1.7724609375E-1-p[2]*6.005859375E-2+p[3]*1.220703125E-2)*2.0+sin(t30)*(p[0]*1.1279296875E-1+p[1]*4.3505859375E-1+p[2]*1.9775390625E-1+p[3]*4.39453125E-3)*2.0+sin(t32)*(p[0]*8.7615966796875E-2+p[1]*5.6488037109375E-2-p[2]*2.4261474609375E-2+p[3]*5.157470703125E-3)*4.0+sin(t34)*(p[0]*1.24237060546875E-1+p[1]*3.04046630859375E-1-p[2]*6.6741943359375E-2+p[3]*1.3458251953125E-2)*4.0+sin(t36)*(p[0]*1.11541748046875E-1+p[1]*4.41741943359375E-1+p[2]*7.0953369140625E-2+p[3]*7.62939453125E-4)*4.0+sin(t38)*(p[0]*1.19842529296875E-1+p[1]*3.99261474609375E-1+p[2]*3.18511962890625E-1+p[3]*3.7384033203125E-2)*4.0)*(1.0/2.4E1)+1.0/2.4E1)*-2.0;
        grad[2] = t0

        return grad

    def fyf(self, p):
        t2 = p[0]*(1.1E1/2.0);
        t3 = p[1]*9.0;
        t4 = p[2]*(9.0/2.0);
        t5 = p[0]*(9.0/2.0);
        t6 = p[1]*(2.7E1/2.0);
        t7 = p[2]*(2.7E1/2.0);
        t8 = p[3]*(9.0/2.0);
        t9 = t5-t6+t7-t8;
        t10 = p[0]*9.0;
        t11 = p[1]*(4.5E1/2.0);
        t12 = p[2]*1.8E1;
        t13 = t8-t10+t11-t12;
        t14 = p[3]-t2+t3-t4;
        t15 = self._yf-p[4]*(sin(p[0]*p[4]-p[4]*t9*(1.0/4.0)-p[4]*t13*(1.0/3.0)+p[4]*t14*(1.0/2.0))+sin(p[0]*p[4]*(1.0/2.0)-p[4]*t9*(1.0/6.4E1)-p[4]*t13*(1.0/2.4E1)+p[4]*t14*(1.0/8.0))*2.0+sin(p[0]*p[4]*(3.0/4.0)-p[4]*t9*7.91015625E-2-p[4]*t13*(9.0/6.4E1)+p[4]*t14*(9.0/3.2E1))*2.0+sin(p[0]*p[4]*(1.0/4.0)-p[4]*t9*9.765625E-4-p[4]*t13*(1.0/1.92E2)+p[4]*t14*(1.0/3.2E1))*2.0+sin(p[0]*p[4]*(3.0/8.0)-p[4]*t9*4.94384765625E-3-p[4]*t13*(9.0/5.12E2)+p[4]*t14*(9.0/1.28E2))*4.0+sin(p[0]*p[4]*(1.0/8.0)-p[4]*t9*6.103515625E-5-p[4]*t13*6.510416666666667E-4+p[4]*t14*(1.0/1.28E2))*4.0+sin(p[0]*p[4]*(5.0/8.0)-p[4]*t9*3.814697265625E-2-p[4]*t13*8.138020833333333E-2+p[4]*t14*(2.5E1/1.28E2))*4.0+sin(p[0]*p[4]*(7.0/8.0)-p[4]*t9*1.4654541015625E-1-p[4]*t13*2.233072916666667E-1+p[4]*t14*(4.9E1/1.28E2))*4.0)*(1.0/2.4E1);
        t0 = t15*t15;
        return t0

    def fyf_grad(self, p):
        grad = [0.0, 0.0, 0.0]

        t2 = p[0]*(1.1E1/2.0);
        t3 = p[1]*9.0;
        t4 = p[2]*(9.0/2.0);
        t5 = p[0]*(9.0/2.0);
        t6 = p[1]*(2.7E1/2.0);
        t7 = p[2]*(2.7E1/2.0);
        t8 = p[3]*(9.0/2.0);
        t9 = t5-t6+t7-t8;
        t10 = p[0]*9.0;
        t11 = p[1]*(4.5E1/2.0);
        t12 = p[2]*1.8E1;
        t13 = t8-t10+t11-t12;
        t14 = p[3]-t2+t3-t4;
        t15 = p[0]*p[4];
        t16 = p[0]*p[4]*(1.0/2.0);
        t17 = p[0]*p[4]*(3.0/4.0);
        t18 = p[0]*p[4]*(1.0/4.0);
        t19 = p[0]*p[4]*(3.0/8.0);
        t20 = p[0]*p[4]*(1.0/8.0);
        t21 = p[0]*p[4]*(5.0/8.0);
        t22 = p[0]*p[4]*(7.0/8.0);
        t23 = p[4]*t14*(1.0/2.0);
        t24 = t15+t23-p[4]*t9*(1.0/4.0)-p[4]*t13*(1.0/3.0);
        t25 = p[4]*t14*(1.0/8.0);
        t26 = t16+t25-p[4]*t9*(1.0/6.4E1)-p[4]*t13*(1.0/2.4E1);
        t27 = p[4]*t14*(9.0/3.2E1);
        t28 = t17+t27-p[4]*t9*7.91015625E-2-p[4]*t13*(9.0/6.4E1);
        t29 = p[4]*t14*(1.0/3.2E1);
        t30 = t18+t29-p[4]*t9*9.765625E-4-p[4]*t13*(1.0/1.92E2);
        t31 = p[4]*t14*(9.0/1.28E2);
        t32 = t19+t31-p[4]*t9*4.94384765625E-3-p[4]*t13*(9.0/5.12E2);
        t33 = p[4]*t14*(1.0/1.28E2);
        t34 = t20+t33-p[4]*t9*6.103515625E-5-p[4]*t13*6.510416666666667E-4;
        t35 = p[4]*t14*(2.5E1/1.28E2);
        t36 = t21+t35-p[4]*t9*3.814697265625E-2-p[4]*t13*8.138020833333333E-2;
        t37 = p[4]*t14*(4.9E1/1.28E2);
        t38 = t22+t37-p[4]*t9*1.4654541015625E-1-p[4]*t13*2.233072916666667E-1;
        t0 = p[4]*(self._yf-p[4]*(sin(t24)+sin(t26)*2.0+sin(t28)*2.0+sin(t30)*2.0+sin(t32)*4.0+sin(t34)*4.0+sin(t36)*4.0+sin(t38)*4.0)*(1.0/2.4E1))*(p[4]*cos(t24)*(3.0/8.0)+p[4]*cos(t26)*(5.1E1/6.4E1)+p[4]*cos(t28)*8.701171875E-1+p[4]*cos(t30)*3.544921875E-1+p[4]*cos(t32)*1.2161865234375+p[4]*cos(t34)*2.259521484375E-1+p[4]*cos(t36)*1.7669677734375+p[4]*cos(t38)*1.5970458984375)*(-1.0/1.2E1);
        grad[0] = t0


        t2 = p[0]*(1.1E1/2.0);
        t3 = p[1]*9.0;
        t4 = p[2]*(9.0/2.0);
        t5 = p[0]*(9.0/2.0);
        t6 = p[1]*(2.7E1/2.0);
        t7 = p[2]*(2.7E1/2.0);
        t8 = p[3]*(9.0/2.0);
        t9 = t5-t6+t7-t8;
        t10 = p[0]*9.0;
        t11 = p[1]*(4.5E1/2.0);
        t12 = p[2]*1.8E1;
        t13 = t8-t10+t11-t12;
        t14 = p[3]-t2+t3-t4;
        t15 = p[0]*p[4];
        t16 = p[0]*p[4]*(1.0/2.0);
        t17 = p[4]*t14*(1.0/8.0);
        t18 = t16+t17-p[4]*t9*(1.0/6.4E1)-p[4]*t13*(1.0/2.4E1);
        t19 = p[0]*p[4]*(3.0/4.0);
        t20 = p[0]*p[4]*(1.0/4.0);
        t21 = p[4]*t14*(1.0/3.2E1);
        t22 = t20+t21-p[4]*t9*9.765625E-4-p[4]*t13*(1.0/1.92E2);
        t23 = p[0]*p[4]*(3.0/8.0);
        t24 = p[4]*t14*(9.0/1.28E2);
        t25 = t23+t24-p[4]*t9*4.94384765625E-3-p[4]*t13*(9.0/5.12E2);
        t26 = p[0]*p[4]*(1.0/8.0);
        t27 = p[4]*t14*(1.0/1.28E2);
        t28 = t26+t27-p[4]*t9*6.103515625E-5-p[4]*t13*6.510416666666667E-4;
        t29 = p[0]*p[4]*(5.0/8.0);
        t30 = p[0]*p[4]*(7.0/8.0);
        t31 = p[4]*t14*(1.0/2.0);
        t32 = t15+t31-p[4]*t9*(1.0/4.0)-p[4]*t13*(1.0/3.0);
        t33 = p[4]*t14*(9.0/3.2E1);
        t34 = t19+t33-p[4]*t9*7.91015625E-2-p[4]*t13*(9.0/6.4E1);
        t35 = p[4]*t14*(2.5E1/1.28E2);
        t36 = t29+t35-p[4]*t9*3.814697265625E-2-p[4]*t13*8.138020833333333E-2;
        t37 = p[4]*t14*(4.9E1/1.28E2);
        t38 = t30+t37-p[4]*t9*1.4654541015625E-1-p[4]*t13*2.233072916666667E-1;
        t0 = p[4]*(self._yf-p[4]*(sin(t18)*2.0+sin(t22)*2.0+sin(t25)*4.0+sin(t28)*4.0+sin(t32)+sin(t34)*2.0+sin(t36)*4.0+sin(t38)*4.0)*(1.0/2.4E1))*(p[4]*cos(t18)*(3.0/6.4E1)+p[4]*cos(t22)*1.201171875E-1+p[4]*cos(t25)*2.669677734375E-1+p[4]*cos(t28)*9.70458984375E-2-p[4]*cos(t32)*(3.0/8.0)-p[4]*cos(t34)*3.955078125E-1-p[4]*cos(t36)*2.838134765625E-1-p[4]*cos(t38)*1.2740478515625)*(1.0/1.2E1);
        grad[1] = t0

        t2 = p[0]*(1.1E1/2.0);
        t3 = p[1]*9.0;
        t4 = p[2]*(9.0/2.0);
        t5 = p[0]*(9.0/2.0);
        t6 = p[1]*(2.7E1/2.0);
        t7 = p[2]*(2.7E1/2.0);
        t8 = p[3]*(9.0/2.0);
        t9 = t5-t6+t7-t8;
        t10 = p[0]*9.0;
        t11 = p[1]*(4.5E1/2.0);
        t12 = p[2]*1.8E1;
        t13 = t8-t10+t11-t12;
        t14 = p[3]-t2+t3-t4;
        t15 = p[0]*p[4];
        t16 = p[0]*p[4]*(1.0/2.0);
        t17 = p[0]*p[4]*(3.0/4.0);
        t18 = p[0]*p[4]*(1.0/4.0);
        t19 = p[0]*p[4]*(3.0/8.0);
        t20 = p[0]*p[4]*(1.0/8.0);
        t21 = p[0]*p[4]*(5.0/8.0);
        t22 = p[0]*p[4]*(7.0/8.0);
        t23 = p[4]*(p[3]-t2+t3-t4)*(1.0/2.0);
        t39 = p[4]*t9*(1.0/4.0);
        t40 = p[4]*t13*(1.0/3.0);
        t24 = t15+t23-t39-t40;
        t25 = p[4]*(p[3]-t2+t3-t4)*(1.0/8.0);
        t41 = p[4]*t9*(1.0/6.4E1);
        t42 = p[4]*t13*(1.0/2.4E1);
        t26 = t16+t25-t41-t42;
        t27 = p[4]*(p[3]-t2+t3-t4)*(1.0/3.2E1);
        t45 = p[4]*t9*9.765625E-4;
        t46 = p[4]*t13*(1.0/1.92E2);
        t28 = t18+t27-t45-t46;
        t29 = p[4]*(p[3]-t2+t3-t4)*(9.0/3.2E1);
        t43 = p[4]*t9*7.91015625E-2;
        t44 = p[4]*t13*(9.0/6.4E1);
        t30 = t17+t29-t43-t44;
        t31 = p[4]*(p[3]-t2+t3-t4)*(1.0/1.28E2);
        t49 = p[4]*t9*6.103515625E-5;
        t50 = p[4]*t13*6.510416666666667E-4;
        t32 = t20+t31-t49-t50;
        t33 = p[4]*(p[3]-t2+t3-t4)*(9.0/1.28E2);
        t47 = p[4]*t9*4.94384765625E-3;
        t48 = p[4]*t13*(9.0/5.12E2);
        t34 = t19+t33-t47-t48;
        t35 = p[4]*(p[3]-t2+t3-t4)*(2.5E1/1.28E2);
        t51 = p[4]*t9*3.814697265625E-2;
        t52 = p[4]*t13*8.138020833333333E-2;
        t36 = t21+t35-t51-t52;
        t37 = p[4]*(p[3]-t2+t3-t4)*(4.9E1/1.28E2);
        t53 = p[4]*t9*1.4654541015625E-1;
        t54 = p[4]*t13*2.233072916666667E-1;
        t38 = t22+t37-t53-t54;
        t0 = (self._yf-p[4]*(sin(t15-t39-t40+p[4]*t14*(1.0/2.0))+sin(t16-t41-t42+p[4]*t14*(1.0/8.0))*2.0+sin(t18-t45-t46+p[4]*t14*(1.0/3.2E1))*2.0+sin(t17-t43-t44+p[4]*t14*(9.0/3.2E1))*2.0+sin(t20-t49-t50+p[4]*t14*(1.0/1.28E2))*4.0+sin(t19-t47-t48+p[4]*t14*(9.0/1.28E2))*4.0+sin(t21-t51-t52+p[4]*t14*(2.5E1/1.28E2))*4.0+sin(t22-t53-t54+p[4]*t14*(4.9E1/1.28E2))*4.0)*(1.0/2.4E1))*(sin(t24)*(1.0/2.4E1)+sin(t26)*(1.0/1.2E1)+sin(t28)*(1.0/1.2E1)+sin(t30)*(1.0/1.2E1)+sin(t32)*(1.0/6.0)+sin(t34)*(1.0/6.0)+sin(t36)*(1.0/6.0)+sin(t38)*(1.0/6.0)+p[4]*(cos(t24)*(p[0]*(1.0/8.0)+p[1]*(3.0/8.0)+p[2]*(3.0/8.0)+p[3]*(1.0/8.0))+cos(t26)*(p[0]*(1.5E1/1.28E2)+p[1]*(5.1E1/1.28E2)-p[2]*(3.0/1.28E2)+p[3]*(1.0/1.28E2))*2.0+cos(t28)*(p[0]*1.2060546875E-1+p[1]*1.7724609375E-1-p[2]*6.005859375E-2+p[3]*1.220703125E-2)*2.0+cos(t30)*(p[0]*1.1279296875E-1+p[1]*4.3505859375E-1+p[2]*1.9775390625E-1+p[3]*4.39453125E-3)*2.0+cos(t32)*(p[0]*8.7615966796875E-2+p[1]*5.6488037109375E-2-p[2]*2.4261474609375E-2+p[3]*5.157470703125E-3)*4.0+cos(t34)*(p[0]*1.24237060546875E-1+p[1]*3.04046630859375E-1-p[2]*6.6741943359375E-2+p[3]*1.3458251953125E-2)*4.0+cos(t36)*(p[0]*1.11541748046875E-1+p[1]*4.41741943359375E-1+p[2]*7.0953369140625E-2+p[3]*7.62939453125E-4)*4.0+cos(t38)*(p[0]*1.19842529296875E-1+p[1]*3.99261474609375E-1+p[2]*3.18511962890625E-1+p[3]*3.7384033203125E-2)*4.0)*(1.0/2.4E1))*-2.0;
        grad[2] = t0

        return grad

    def ftf(self, p):
      t2 = self._tf-p[0]*p[4]+p[4]*(p[0]*(1.1E1/2.0)-p[1]*9.0+p[2]*(9.0/2.0)-p[3])*(1.0/2.0)+p[4]*(p[0]*(9.0/2.0)-p[1]*(2.7E1/2.0)+p[2]*(2.7E1/2.0)-p[3]*(9.0/2.0))*(1.0/4.0)-p[4]*(p[0]*9.0-p[1]*(4.5E1/2.0)+p[2]*1.8E1-p[3]*(9.0/2.0))*(1.0/3.0);
      t0 = t2*t2;
      return t0

    def ftf_grad(self, p):
        grad = [0.0, 0.0, 0.0]

        t0 = p[4]*(self._tf-p[0]*p[4]+p[4]*(p[0]*(1.1E1/2.0)-p[1]*9.0+p[2]*(9.0/2.0)-p[3])*(1.0/2.0)+p[4]*(p[0]*(9.0/2.0)-p[1]*(2.7E1/2.0)+p[2]*(2.7E1/2.0)-p[3]*(9.0/2.0))*(1.0/4.0)-p[4]*(p[0]*9.0-p[1]*(4.5E1/2.0)+p[2]*1.8E1-p[3]*(9.0/2.0))*(1.0/3.0))*(-3.0/4.0);
        grad[0] = t0

        t0 = p[4]*(self._tf-p[0]*p[4]+p[4]*(p[0]*(1.1E1/2.0)-p[1]*9.0+p[2]*(9.0/2.0)-p[3])*(1.0/2.0)+p[4]*(p[0]*(9.0/2.0)-p[1]*(2.7E1/2.0)+p[2]*(2.7E1/2.0)-p[3]*(9.0/2.0))*(1.0/4.0)-p[4]*(p[0]*9.0-p[1]*(4.5E1/2.0)+p[2]*1.8E1-p[3]*(9.0/2.0))*(1.0/3.0))*(-3.0/4.0);
        grad[1] = t0

        t0 = (p[0]*(1.0/8.0)+p[1]*(3.0/8.0)+p[2]*(3.0/8.0)+p[3]*(1.0/8.0))*(self._tf-p[0]*p[4]+p[4]*(p[0]*(1.1E1/2.0)-p[1]*9.0+p[2]*(9.0/2.0)-p[3])*(1.0/2.0)+p[4]*(p[0]*(9.0/2.0)-p[1]*(2.7E1/2.0)+p[2]*(2.7E1/2.0)-p[3]*(9.0/2.0))*(1.0/4.0)-p[4]*(p[0]*9.0-p[1]*(4.5E1/2.0)+p[2]*1.8E1-p[3]*(9.0/2.0))*(1.0/3.0))*-2.0;
        grad[2] = t0

        return grad

    def fbe(self, p):
        t0 = p[4]*(p[0]*p[1]*9.9E1-p[0]*p[2]*3.6E1+p[0]*p[3]*1.9E1-p[1]*p[2]*8.1E1-p[1]*p[3]*3.6E1+p[2]*p[3]*9.9E1+(p[0]*p[0])*6.4E1+(p[1]*p[1])*3.24E2+(p[2]*p[2])*3.24E2+(p[3]*p[3])*6.4E1)*(1.0/8.4E2);
        return t0

    def fbe_grad(self, p):
        grad = [0.0, 0.0, 0.0]

        t0 = p[4]*(p[0]*9.9E1+p[1]*6.48E2-p[2]*8.1E1-p[3]*3.6E1)*(1.0/8.4E2);
        grad[0] = t0

        t0 = p[4]*(p[0]*3.6E1+p[1]*8.1E1-p[2]*6.48E2-p[3]*9.9E1)*(-1.0/8.4E2);
        grad[1] = t0

        t0 = p[0]*p[1]*(3.3E1/2.8E2)-p[0]*p[2]*(3.0/7.0E1)+p[0]*p[3]*(1.9E1/8.4E2)-p[1]*p[2]*(2.7E1/2.8E2)-p[1]*p[3]*(3.0/7.0E1)+p[2]*p[3]*(3.3E1/2.8E2)+(p[0]*p[0])*(8.0/1.05E2)+(p[1]*p[1])*(2.7E1/7.0E1)+(p[2]*p[2])*(2.7E1/7.0E1)+(p[3]*p[3])*(8.0/1.05E2);
        grad[2] = t0

        return grad

_path_optimizer = PathOptimizer()


In [ ]:
# V1.0 maneuver static car(fixed mechanism)

import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
client = carla.Client('localhost', 2000)
world = client.get_world()
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla/')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)


# define speed contstants
PREFERRED_SPEED = 30 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 40

def collision_check(point, obstacles):
    collision_free = True
    for k in range(len(obstacles)):
        collision_dist=math.sqrt((point[0]-obstacles[k][0])**2+(point[1]-obstacles[k][1])**2)
        if collision_dist<1.51:
            collision_free=False
            break

    return collision_free
    
def semantic_lidar_data(point_cloud_data,car,l):
    l.clear()
    for detection in point_cloud_data:
        l.append([detection.point.x+car.get_location().x,detection.point.y+car.get_location().y])

def generate_lidar_blueprint(blueprint_library):
    lidar_blueprint = blueprint_library.find('sensor.lidar.ray_cast_semantic')
    lidar_blueprint.set_attribute('channels', str(32))
    lidar_blueprint.set_attribute("points_per_second",str(56000))
    lidar_blueprint.set_attribute("rotation_frequency",str(10))
    lidar_blueprint.set_attribute("range",str(15))
    lidar_blueprint.set_attribute("upper_fov",str(5))
    lidar_blueprint.set_attribute("lower_fov",str(-5))
    return lidar_blueprint


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

#function to subtract 2 vectors
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.x
    wp_y = wp.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

wa=world.get_map().get_waypoint(carla.Location(x=-60,y=137,z=0.5))
start_point=wa.transform
start_point.location.z=0.5
midle_point=wa.transform
midle_point.location.z=0.5
midle_point.location.x=0

vehicle_bp = world.get_blueprint_library().filter('*mini*')
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
vehicle2 = world.try_spawn_actor(vehicle_bp[0], midle_point)

get_blueprint_of_world = world.get_blueprint_library()
lidar_sensor = generate_lidar_blueprint(get_blueprint_of_world)
sensor_lidar_spawn_point = carla.Transform(carla.Location(x=0, y=0, z=2.0),
                                           carla.Rotation(pitch=0.000000, yaw=0.0, roll=0.000000))
sensor = world.spawn_actor(lidar_sensor, sensor_lidar_spawn_point, attach_to=vehicle)
wa=world.get_map().get_waypoint(carla.Location(x=60,y=137,z=0.5))
end_point=wa.transform.location

route = grp.trace_route(start_point.location, end_point)
for waypoint in route:
    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
        color=carla.Color(r=0, g=0, b=255), life_time=30.0,
        persistent_lines=True) 
curIndx=1
flag=0
path = _path_optimizer.optimize_spiral(10,0,0)
y=[y+vehicle.get_location().y for y in list(path[1][::2])]
x=[x+vehicle.get_location().x for x in list(path[0][::2])]
for i in range(len(x)):
    world.debug.draw_string(carla.Location(x=x[i],y=y[i],z=0), '^', draw_shadow=False,
            color=carla.Color(r=0, g=0, b=255), life_time=30.0,
            persistent_lines=True)

curr_wp = 1
turn_flag=0
turn_flag2=0

ladarData=[]
sensor.listen(lambda point_cloud_data: semantic_lidar_data(point_cloud_data,vehicle,ladarData))
while curr_wp<len(route)-11:
    ladarData1=ladarData.copy()
    if collision_check([route[curr_wp+10][0].transform.location.x,route[curr_wp+10][0].transform.location.y],ladarData)==False and turn_flag2 ==0:
        turn_flag=1
        turn_flag2=1
        curIndx=1

    if flag ==0 and turn_flag==1:
        
        path = _path_optimizer.optimize_spiral(15,-3,0)
        y=[y+vehicle.get_location().y for y in list(path[1][::2])]
        x=[x+vehicle.get_location().x for x in list(path[0][::2])]
        for i in range(len(x)):
            world.debug.draw_string(carla.Location(x=x[i],y=y[i],z=0), '^', draw_shadow=False,
                color=carla.Color(r=0, g=0, b=255), life_time=30.0,
                persistent_lines=True)

        flag=1
        curIndx=1

    elif flag ==1 and curIndx==len(x)-1 and turn_flag==1:
        path = _path_optimizer.optimize_spiral(15,3,0)
        y=[y+vehicle.get_location().y for y in list(path[1][::2])]
        x=[x+vehicle.get_location().x for x in list(path[0][::2])]
        for i in range(len(x)):
            world.debug.draw_string(carla.Location(x=x[i],y=y[i],z=0), '^', draw_shadow=False,
                color=carla.Color(r=0, g=0, b=255), life_time=30.0,
                persistent_lines=True)
        
        curIndx=1
        turn_flag=0


    elif turn_flag==0 and turn_flag2==0:
        path = _path_optimizer.optimize_spiral(10,0,0)
        y=[y+vehicle.get_location().y for y in list(path[1][::2])]
        x=[x+vehicle.get_location().x for x in list(path[0][::2])]
        for i in range(len(x)):
            world.debug.draw_string(carla.Location(x=x[i],y=y[i],z=0), '^', draw_shadow=False,
                color=carla.Color(r=0, g=0, b=255), life_time=30.0,
                persistent_lines=True)
        curIndx=1

    if curIndx==len(x)-1 and flag==1:
        turn_flag2=0
        flag=0
    world.tick()

    while curr_wp<len(route) and carla.Location(x=x[curIndx],y=route[curr_wp][0].transform.location.y,z=0).distance(route[curr_wp][0].transform.location)<1:
        curr_wp +=1 #move to next wp if we are too close
    while curIndx<len(x)-1 and vehicle.get_transform().location.distance(carla.Location(x=x[curIndx],y=y[curIndx],z=0))<1:
         curIndx +=1
    
    predicted_angle = get_angle(vehicle,carla.Location(x=x[curIndx],y=y[curIndx],z=0))
    v = vehicle.get_velocity()
    speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
    estimated_throttle = maintain_speed(speed)
    # extra checks on predicted angle when values close to 360 degrees are returned
    if predicted_angle<-300:
        predicted_angle = predicted_angle+360
    elif predicted_angle > 300:
        predicted_angle = predicted_angle -360
    steer_input = predicted_angle
    # limit steering to max angel, say 40 degrees
    if predicted_angle<-MAX_STEER_DEGREES:
        steer_input = -MAX_STEER_DEGREES
    elif predicted_angle>MAX_STEER_DEGREES:
        steer_input = MAX_STEER_DEGREES
    # conversion from degrees to -1 to +1 input for apply control function
    steer_input = steer_input/75
    
    vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))


for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()


In [ ]:
# V1.2 try fixed trajictories(todo:fix small trajs and get aaway from the car next me )

import carla #the sim library itself
import time # to set a delay after each photo
import numpy as np #in this example to change image representation - re-shaping
import math
import sys
client = carla.Client('localhost', 2000)
world = client.get_world()
sys.path.append('/home/youmad55/Downloads/CARLA_0.9.15/PythonAPI/carla/')
from agents.navigation.global_route_planner import GlobalRoutePlanner
grp = GlobalRoutePlanner(world.get_map(), 1)


# define speed contstants
PREFERRED_SPEED = 10 # what it says
SPEED_THRESHOLD = 2 #defines when we get close to desired speed so we drop the

# Max steering angle
MAX_STEER_DEGREES = 40

trajs=[]
for ys in range(-6,7):
    path = _path_optimizer.optimize_spiral(13,ys/2,0)
    y=list(path[1][::2])
    x=list(path[0][::2])
    trajs.append([x,y])
def best_trajectory(vehicle,curr_wp,ladarData):
    bestTraj=[]
    ys=6
    minCost=float('inf')
    for i in range(13):
        y=[y+vehicle.get_location().y for y in trajs[i][1]]
        x=[x+vehicle.get_location().x for x in trajs[i][0]]
        #for i in range(len(x)):
            #world.debug.draw_string(carla.Location(x=x[i],y=y[i],z=0), '^', draw_shadow=False,
            #    color=carla.Color(r=0, g=0, b=255), life_time=30.0,
            #    persistent_lines=True)
            #print(ys)
        cost=0
        cost+=50*carla.Location(x=x[-1],y=y[-1],z=0).distance(carla.Location(x=x[-1],y=curr_wp.transform.location.y,z=0))
        #if collision_check([x[4],y[4]],ladarData)==False  or collision_check([x[9],y[9]],ladarData)==False  or collision_check([x[14],y[14]],ladarData)==False  or collision_check([x[19],y[19]],ladarData)==False or collision_check([x[20],y[20]],ladarData)==False:
        if collision_check([x[-1],y[-1]],ladarData)==False or collision_check([x[4],y[4]],ladarData)==False  or collision_check([x[9],y[9]],ladarData)==False  or collision_check([x[14],y[14]],ladarData)==False  or collision_check([x[19],y[19]],ladarData)==False or collision_check([x[20],y[20]],ladarData)==False:
        #if collision_check([x[-1],y[-1]],ladarData)==False or collision_check([x[7],y[7]],ladarData)==False or collision_check([x[15],y[15]],ladarData)==False:
           cost= float('inf')
        if cost<minCost:
            minCost=cost
            bestTraj=[x,y]
            ys=i
        #print(cost)
    return ys,bestTraj

def collision_check(point, obstacles):
    collision_free = True
    for k in range(len(obstacles)):
        collision_dist=math.sqrt((point[0]-obstacles[k][0])**2+(point[1]-obstacles[k][1])**2)
        if collision_dist<1.4:
            collision_free=False
            break

    return collision_free


def lidar_callback(point_cloud, point_list):
    
    data = np.copy(np.frombuffer(point_cloud.raw_data, dtype=np.dtype('f4')))
    data = np.reshape(data, (int(data.shape[0] / 4), 4))
    points = data[:, :-1]
    points[:, :1] = -points[:, :1]

def semantic_lidar_data(point_cloud_data,car,l):
    l.clear()
    c=math.cos(car.get_transform().rotation.yaw*math.pi/180)
    s=math.sin(car.get_transform().rotation.yaw*math.pi/180)
    for detection in point_cloud_data:
        #l.append([round(detection.point.x+car.get_location().x,1),round(detection.point.y+car.get_location().y,1)])
        #l.append([detection.point.x+car.get_location().x,detection.point.y+car.get_location().y])
        l.append([round((detection.point.x *c +detection.point.y*-s)+car.get_location().x,1),round((detection.point.x*s+detection.point.y*c)+car.get_location().y,1)])

def generate_lidar_blueprint(blueprint_library):
    lidar_blueprint = blueprint_library.find('sensor.lidar.ray_cast')
    lidar_blueprint.set_attribute('channels', str(32))
    lidar_blueprint.set_attribute("points_per_second",str(56000))
    lidar_blueprint.set_attribute("rotation_frequency",str(1000))
    lidar_blueprint.set_attribute("range",str(50))
    lidar_blueprint.set_attribute("upper_fov",str(1))
    lidar_blueprint.set_attribute("lower_fov",str(-1))
#    lidar_blueprint.set_attribute("horizontal_fov",str(360))
    return lidar_blueprint


# maintain speed function
def maintain_speed(s):
    ''' 
    this is a very simple function to maintan desired speed
    s arg is actual current speed
    '''
    if s >= PREFERRED_SPEED:
        return 0
    elif s < PREFERRED_SPEED - SPEED_THRESHOLD:
        return 0.9 # think of it as % of "full gas"
    else:
        return 0.4 # tweak this if the car is way over or under preferred speed 

#function to subtract 2 vectors
def angle_between(v1, v2):
    return math.degrees(np.arctan2(v1[1], v1[0]) - np.arctan2(v2[1], v2[0]))

# function to get angle between the car and target waypoint
def get_angle(car,wp):
    '''
    this function to find direction to selected waypoint
    '''
    vehicle_pos = car.get_transform()
    car_x = vehicle_pos.location.x
    car_y = vehicle_pos.location.y
    wp_x = wp.x
    wp_y = wp.y
    
    # vector to waypoint
    x = (wp_x - car_x)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    y = (wp_y - car_y)/((wp_y - car_y)**2 + (wp_x - car_x)**2)**0.5
    
    #car vector
    car_vector = vehicle_pos.get_forward_vector()
    degrees = angle_between((x,y),(car_vector.x,car_vector.y))

    return degrees

wa=world.get_map().get_waypoint(carla.Location(x=-60,y=137,z=0.5))
start_point=wa.transform
start_point.location.z=0.5
midle_point=wa.transform
midle_point.location.z=0.5
midle_point.location.x=0
midle_point2=wa.transform
midle_point2.location.z=0.5
midle_point2.location.x=-50
midle_point2.location.y=midle_point2.location.y+3
vehicle_bp = world.get_blueprint_library().filter('*mini*')
vehicle = world.try_spawn_actor(vehicle_bp[0], start_point)
vehicle2 = world.try_spawn_actor(vehicle_bp[0], midle_point)
#vehicle3 = world.try_spawn_actor(vehicle_bp[0], midle_point2)

get_blueprint_of_world = world.get_blueprint_library()
lidar_sensor = generate_lidar_blueprint(get_blueprint_of_world)
sensor_lidar_spawn_point = carla.Transform(carla.Location(x=0, y=0, z=1.2),
                                           carla.Rotation(pitch=0.000000, yaw=0.0, roll=0.000000))
sensor = world.spawn_actor(lidar_sensor, sensor_lidar_spawn_point, attach_to=vehicle)
wa=world.get_map().get_waypoint(carla.Location(x=60,y=137,z=0.5))
end_point=wa.transform.location

route = grp.trace_route(start_point.location, end_point)
#for waypoint in route:
#    world.debug.draw_string(waypoint[0].transform.location, '^', draw_shadow=False,
#        color=carla.Color(r=0, g=0, b=255), life_time=30.0,
#        persistent_lines=True) 
curIndx=1

curr_wp = 1

ladarData=[]
sensor.listen(lambda point_cloud_data: semantic_lidar_data(point_cloud_data,vehicle,ladarData))
#l=[]
#l=ladarData.copy()

l=set()
ladarData1=ladarData.copy()
s={tuple(ladarData1[i]) for i in range(len(ladarData1))}
l=l.union(s)
l=list(l)

ys,tr=best_trajectory(vehicle,route[10][0],l)
print(ys)
y=tr[1]
x=tr[0]

straight=True

c=0
f=0
while curr_wp<len(route)-11:
#    time.sleep(0.001)
    c+=1
    f+=1
#    l.clear()
#    l=ladarData.copy()

    l=set(l)
    ladarData1=ladarData.copy()
    s={tuple(ladarData1[i]) for i in range(len(ladarData1))}
    l=l.union(s)
    l=list(l)

    xf=len(l)
    for i in range(xf):
        if -l[xf-1-i][0]+vehicle.get_location().x>3 or abs(l[xf-1-i][1]-vehicle.get_location().y)>7:
            l.pop(xf-1-i)
    if c==50:
        c=0
        for i in l:
            world.debug.draw_string(carla.Location(x=i[0],y=i[1],z=0), '^', draw_shadow=False,
                color=carla.Color(r=0, g=0, b=255), life_time=10.0,
                persistent_lines=True)
        
    if ys==6 and straight==True:
        ys,tr=best_trajectory(vehicle,route[curr_wp+5][0],l)
        curIndx=1
        #print("i")
    else:
        straight=False
    if tr:
        y=tr[1]
        x=tr[0]
        for i in range(len(x)):
            world.debug.draw_string(carla.Location(x=x[i],y=y[i],z=0), '^', draw_shadow=False,
                color=carla.Color(r=0, g=0, b=255), life_time=1.0,
                persistent_lines=True)
    
        world.tick()
    
        while curr_wp<len(route) and carla.Location(x=x[curIndx],y=route[curr_wp][0].transform.location.y,z=0).distance(route[curr_wp][0].transform.location)<1:
            curr_wp +=1 #move to next wp if we are too close
        while curIndx<len(x)-1 and vehicle.get_transform().location.distance(carla.Location(x=x[curIndx],y=y[curIndx],z=0))<1:
             curIndx +=1
        
        predicted_angle = get_angle(vehicle,carla.Location(x=x[curIndx],y=y[curIndx],z=0))
        v = vehicle.get_velocity()
        speed = round(3.6 * math.sqrt(v.x**2 + v.y**2 + v.z**2),0)
        estimated_throttle = maintain_speed(speed)
        # extra checks on predicted angle when values close to 360 degrees are returned
        if predicted_angle<-300:
            predicted_angle = predicted_angle+360
        elif predicted_angle > 300:
            predicted_angle = predicted_angle -360
        steer_input = predicted_angle
        # limit steering to max angel, say 40 degrees
        if predicted_angle<-MAX_STEER_DEGREES:
            steer_input = -MAX_STEER_DEGREES
        elif predicted_angle>MAX_STEER_DEGREES:
            steer_input = MAX_STEER_DEGREES
        # conversion from degrees to -1 to +1 input for apply control function
        steer_input = steer_input/75
        
        vehicle.apply_control(carla.VehicleControl(throttle=estimated_throttle, steer=steer_input))
        #print(f)
        if curIndx>=len(x)-1:
            straight=True
            ys=6
            #print("h")

for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()


In [ ]:
#up to down

path = _path_optimizer.optimize_spiral(10,3,0)
y=[y+vehicle.get_location().y for y in list(path[0][::2])]
x=[x+vehicle.get_location().x for x in list(path[1][::2])]


In [ ]:
#down to up

path = _path_optimizer.optimize_spiral(10,-3,0)
y=[-y+vehicle.get_location().y for y in list(path[0][::2])]
x=[x+vehicle.get_location().x for x in list(path[1][::2])]


In [ ]:
#left to right

path = _path_optimizer.optimize_spiral(10,-3,0)
y=[y+vehicle.get_location().y for y in list(path[1][::2])]
x=[x+vehicle.get_location().x for x in list(path[0][::2])]


In [ ]:
#right to left

path = _path_optimizer.optimize_spiral(10,3,0)
y=[y+vehicle.get_location().y for y in list(path[1][::2])]
x=[-x+vehicle.get_location().x for x in list(path[0][::2])]
